In [24]:
import spotipy
from spotipy.oauth2 import SpotifyClientCredentials
import pandas as pd
import time 

# DEFINE LOGIN CREDENTIALS
userID         = '1124570809'
client_id      = '9eb4c6ef069845e4bde31c6581daac26'
client_secret  = '9995ec38d7294435acfb7c8fe81b99ec'

client_credentials_manager = SpotifyClientCredentials(client_id, client_secret)
sp = spotipy.Spotify(client_credentials_manager=client_credentials_manager)

# LOAD DATASETS IDS FROM CSV FILE
IDs_df = pd.read_csv("Data/DatasetsIDs.csv",";")

In [23]:
# Load Playlists and Save into Data Folder

def getTrackIDs(username,playlist_id):
    results = sp.user_playlist_tracks(username,playlist_id)
    tracks = results['items']
    while results['next']:
        results = sp.next(results)
        tracks.extend(results['items'])
    IDs = []
    for i in range(len(tracks)):
        IDs.append(tracks[i]["track"]['id']) 
    return IDs


# Function for retrieving track ids
def getTrackIDs_OLD(user, playlist_id):
    ids = []
    playlist = sp.user_playlist(user, playlist_id)
    for item in playlist['tracks']['items']:
        track = item['track']
        ids.append(track['id'])
    return ids

# Function that extracts track features
def getTrackFeatures(id):
    try:
      meta = sp.track(id)
      features = sp.audio_features(id)

      # meta
      name = meta['name']
      album = meta['album']['name']
      artist = meta['album']['artists'][0]['name']
      release_date = meta['album']['release_date']
      length = meta['duration_ms']
      popularity = meta['popularity']

      # features
      acousticness = features[0]['acousticness']
      danceability = features[0]['danceability']
      energy = features[0]['energy']
      instrumentalness = features[0]['instrumentalness']
      liveness = features[0]['liveness']
      loudness = features[0]['loudness']
      speechiness = features[0]['speechiness']
      tempo = features[0]['tempo']
      time_signature = features[0]['time_signature']

      track = [name, album, artist, release_date, length, popularity, acousticness, danceability, energy, instrumentalness, liveness, loudness, speechiness, tempo, time_signature]
      return track
    except:
      return []

# Loop through Datasets
for index, row in IDs_df.iterrows():
    if row['ID'] == 0:
        ID   = row['ID']
        Name = row['Dataset']
        ids = getTrackIDs(userID, ID)

        # loop over track ids 
        tracks = []
        for i in range(len(ids)):
          track = getTrackFeatures(ids[i])
          if len(track) > 0:
              tracks.append(track)

        # create dataset
        df = pd.DataFrame(tracks, columns = ['name', 'album', 'artist', 'release_date', 'length', 'popularity', 'acousticness', 'danceability', 'energy', 'instrumentalness', 'liveness', 'loudness', 'speechiness', 'tempo', 'time_signature'])

        df.to_csv("Data/Datasets/Dataset" + str(Name) + ".csv", sep = ',')

 

In [19]:
# Current Tracks
currentTopSongs = sp.current_user_top_tracks
currentTopSongs

<bound method Spotify.current_user_top_tracks of <spotipy.client.Spotify object at 0x7fc6c00ea390>>
